# InSituParse - Visualize data with napari and crop datasets

In [1]:
## The following code ensures that all functions and init files are reloaded before executions.
%load_ext autoreload
%autoreload 2

In [2]:
from xeniumdata import XeniumData
import xeniumdata
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

## Integrate HE images into Xenium data structure

Idea:
1. Place HE images into folder close to the xenium output files
2. Register images using SIFT algorithm and extract transformation matrix (for Xenium Explorer).
3. Save registered images in Xenium folder and include information about image paths into the `experiment.xenium` file and generate a separate `experiment_he.xenium` file to prevent problems with xenium explorer.

In [3]:
# input directories
data_dir = Path("C:/Users/ge37voy/data/datasets/Xenium_FFPE_Human_Breast_Cancer_Rep1_outs/output-XETG00000__slide_id__sample_id/")

In [30]:
# load xenium data
xd = XeniumData(data_dir)

In [31]:
xd

XeniumData
Slide ID:	slide_id
Region ID:	sample_id
Data path:	C:\Users\ge37voy\data\datasets\Xenium_FFPE_Human_Breast_Cancer_Rep1_outs
Data folder:	output-XETG00000__slide_id__sample_id
Metadata file:	experiment.xenium

In [32]:
xd.read_all()

No folder named `annotations` found. `read_annotations()` skipped.
Running read_boundaries()
Running read_images()
Running read_matrix()
Running read_transcripts()


In [35]:
xd.interactive()

Viewer(camera=Camera(center=(0.0, 12888.5, 17707.5), zoom=0.013626609442060083, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 25778.0, 1.0), (0.0, 35416.0, 1.0)), current_step=(12888, 17707), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'DAPI' at 0x290cf0cd460>, <Points layer 'None' at 0x290cc15a970>], help='use <5> for transform, use <2> for add points, use <3> for select points', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x00000290CA14AAF0>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_gen={}, keymap={})

In [11]:
df = pd.read_parquet(r"C:\Users\ge37voy\data\datasets\Xenium_FFPE_Human_Breast_Cancer_Rep1_outs\output-XETG00000__slide_id__sample_id\nucleus_boundaries.parquet")

In [13]:
df.index.str.decode()

AttributeError: Can only use .str accessor with string values!

In [32]:
df.cell_id

0               1
1               1
2               1
3               1
4               1
            ...  
2180757    167780
2180758    167780
2180759    167780
2180760    167780
2180761    167780
Name: cell_id, Length: 2180762, dtype: int32

In [36]:
df.cell_id.astype(str).str.encode('utf-8')

0               b'1'
1               b'1'
2               b'1'
3               b'1'
4               b'1'
             ...    
2180757    b'167780'
2180758    b'167780'
2180759    b'167780'
2180760    b'167780'
2180761    b'167780'
Name: cell_id, Length: 2180762, dtype: object

In [37]:
from pandas.api.types import is_numeric_dtype, is_string_dtype

In [40]:
is_numeric_dtype(df.cell_id.astype(str))

False

In [ ]:
df.cell_id

In [14]:
l = list("abcd")

In [16]:
l.remove('a')

In [17]:
l

['b', 'c', 'd']

In [21]:
list(xd.path.parent.glob("annotations"))

[]

## Develop cropping from viewer input

In [20]:
xd.crop(shape_layer=None, xlim=(10000, 12000), ylim=(12000, 18000))

In [21]:
xd.interactive()

Viewer(camera=Camera(center=(0.0, 2999.5, 999.5), zoom=0.14037868649318463, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 6000.0, 1.0), (0.0, 2000.951171875, 1.0)), current_step=(2999, 999), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'DAPI' at 0x1f0db3a0820>, <Image layer 'HE' at 0x1f0db3ac100>, <Image layer 'panCK' at 0x1f0e14d5eb0>, <Points layer 'None' at 0x1f0e1094be0>], help='use <5> for transform, use <2> for add points, use <3> for select points', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x000001F0C716B670>], _persisted_mouse

In [30]:
xd.crop(shape_layer="Shapes")

In [31]:
xd

XeniumData
Slide ID:	0003621
Region ID:	Region_1
Data path:	C:\Users\ge37voy\data\2301_CRC\2301-01_20230504\01_xenium_data
Data folder:	output-XETG00050__0003621__Region_1__20230504__121954
Metadata file:	experiment_modified.xenium
    ➤ images
       DAPI:	(1449, 1135)
       HE:	(1449, 1135, 3)
       panCK:	(1449, 1135, 3)
    ➤ matrix
       AnnData object with n_obs × n_vars = 704 × 280
	       obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area'
	       var: 'gene_ids', 'feature_types', 'genome'
	       obsm: 'spatial'
    ➤ transcripts
	   DataFrame with shape 64608 x 10
    ➤ boundaries
       cells
       nuclei
    ➤ annotations
       tanja:	41 annotations, 5 classes ('Tumor', 'Fat', 'Mucus', 'Immune cells', 'Peritumoral Zone') 

In [32]:
xd.interactive()

Viewer(camera=Camera(center=(0.0, 724.0, 567.0), zoom=0.5812782049407231, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 1449.0, 1.0), (0.0, 1136.4420273850646, 1.0)), current_step=(724, 567), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'DAPI' at 0x1f0f2ead7f0>, <Image layer 'HE' at 0x1f0f2ee9250>, <Image layer 'panCK' at 0x1f12a6846a0>, <Points layer 'None' at 0x1f13e99cdc0>], help='use <5> for transform, use <2> for add points, use <3> for select points', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x000001F0C716B670>], _persisted_mous